In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'https://osamigosdopresidentelula.blogspot.com/'

In [3]:
req = requests.get(link)

In [4]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('h3', class_='post-title entry-title')

In [5]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        print(path_image)
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

https://osamigosdopresidentelula.blogspot.com/2018/10/justica-eleitoral-encontra-doacoes-de.html
https://1.bp.blogspot.com/-nHPotZEZnow/W7SyBuc2R_I/AAAAAAAAgOQ/ToG9Tbi5MgsrlU4pNiu2Z0Vx48L7alpIgCLcBGAs/w1200-h630-p-k-no-nu/%252C0000000.jpg
100% [..............................................................................] 14137 / 14137,0000000.jpg
C:\Users\admin\rss3\src\midia_postagem\images\,0000000.jpg
['Justiça Eleitoral encontra doações de empresas disfarçada: Entre os casos estão doações por mortos,desempregados e beneficiários do Bolsa Família']
news_in_db: False
 -- no categories -- 
https://osamigosdopresidentelula.blogspot.com/2018/10/antes-de-sair-temer-que-vender-tudo-que.html
https://1.bp.blogspot.com/-2hwQWihakng/W7StuAzXVCI/AAAAAAAAgOE/Bytg_9ABD98Yr7k6ssQOOyla8uqpoOtZQCLcBGAs/w1200-h630-p-k-no-nu/%252C0000000.jpg
100% [..............................................................................] 15381 / 15381,0000000.jpg
C:\Users\admin\rss3\src\midia_postagem\images\,

KeyboardInterrupt: 